# Introduction to Image query, download and display

**Base:** For Rubin DP1 in the RSP or externally

**LSST data products:** `visit_image`, `deep_coadd`, `template_coadd`, `difference_image`

**SSSC rsp_queries module:** `query_images`

**Packages used:** `lsst.rsp.utils`, `pyvo`, `lsst.rsp.service`, `astropy.time`, `astropy.table`, `astropy.coordinates`, `astropy.units`, [`lsst.afw.display`, `pyds9`]

**Authors:** Originally developed by Tim Lister


## 1. Introduction

This notebook demonstrates how to find images using the `ivoa.ObsCore`, retrieve images from a remote location using the [IVOA datalink](https://www.ivoa.net/documents/DataLink/20211115/WD-DataLink-1.1-20211115.html) protocol and download image cutouts using the [Server-side Operations for Data Access (SODA)](https://www.ivoa.net/documents/SODA/20170517/REC-SODA-1.0.html) protocal.

### 1.1 Import packages
Import the rsp_queries module and some common astronomy packages 

In [40]:
import os
from datetime import datetime

from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord, CustomBarycentricEcliptic

from sso_query.query_images import build_query, make_query, download_data, get_image_cutout, display_image


### 1.2 Connecting to the RSP

Checking for access to the RSP and checking whether `ACCESS_TOKEN` and `EXTERNAL_INSTANCE_URL` have been set if running externally, is handled by `check_rsp_access()`. This is run by the query routines and doesn't need to be done by the user (although it can be;  `check_rsp_access()` will return `True` if access is good)

## 2. Find images

### 2.1 Query building
The first step is to find some images to work with. This is done by `build_query()` (which is called from the query runner `make_query()`). This routine is designed to handle the coordinates, filter bands and times in a (hopefully) SSSC user-friendly way.

We define the center right ascension and declination of our target field which is the low ecliptic latitude 'Rubin SV 38 7' field which has the majority of the Solar System Object (SSO) detections (5304), and unique objects (406).

In [12]:
target_ra = 37.86
target_dec = +6.98
center = SkyCoord(target_ra, target_dec, unit='deg', frame='icrs')
print(f"RA/Dec= {center.to_string('hmsdms', sep=' ')}, ecliptic latitude={ center.transform_to(CustomBarycentricEcliptic).lat:.1f}")

RA/Dec= 02 31 26.4 +06 58 48, ecliptic latitude=-7.5 deg


For times, `make_query()` will take either Modified Julian Dates (MJD) in the TAI timescale as a `float` if you have those from e.g. another tutorial notebook or a `astropy.Time` object in any format and timescale.

(There is 37s difference between UTC and TAI in 2024, hence the difference from .0 above)

In [17]:
time1 = Time(datetime(2024, 11, 27), scale='utc')
time2 = Time(datetime(2024, 12,  9), scale='utc')
print(f"Search Timespan MJD {time1.tai.mjd:.6f} -> {time2.tai.mjd:.6f} TAI")

Search Timespan MJD 60641.000428 -> 60653.000428 TAI


Make and print the ADQL query string. By default the bands to be searched are the most relevant for SSO discovery, namely _g, r, i_ but this can be changed by doing e.g, `bands=['r', 'i', 'z', 'y']` in the call 

In [19]:
query = build_query(center, t_min=time1, t_max=time2)
print(query)

SELECT lsst_visit, lsst_detector, lsst_tract, lsst_patch, lsst_band,s_ra, s_dec, t_min, t_max, s_region, access_url
FROM ivoa.ObsCore
WHERE calib_level = 2
AND CONTAINS(POINT('ICRS', 37.86,6.98), s_region) = 1
AND (lsst_band = 'g' OR lsst_band = 'r' OR lsst_band = 'i')
AND t_min > 60641.00042824074 AND t_max < 60653.00042824074
ORDER BY t_min ASC



To search for difference or coadded images, set `calib_level=3`:

In [21]:
diffs_query = build_query(center, bands=['r', 'i', 'z', 'y'], t_min=60641.0, t_max=60653.0, calib_level=3)
print(diffs_query)

SELECT lsst_visit, lsst_detector, lsst_tract, lsst_patch, lsst_band,s_ra, s_dec, t_min, t_max, s_region, access_url
FROM ivoa.ObsCore
WHERE calib_level = 3
AND CONTAINS(POINT('ICRS', 37.86,6.98), s_region) = 1
AND (lsst_band = 'r' OR lsst_band = 'i' OR lsst_band = 'z' OR lsst_band = 'y')
AND t_min > 60641.0 AND t_max < 60653.0
ORDER BY t_min ASC



### 2.2 Query execution

`make_query()` takes the same options as `build_query()` (which it calls) and checks for access and actually runs the query and returns an `astropy.Table` of results.

In [38]:
results = make_query(center, t_min=time1, t_max=time2)
print(results[('lsst_visit', 'lsst_band', 's_ra', 's_dec')])
## List of all the columns
print(results.colnames)

Executing the following query:
SELECT lsst_visit, lsst_detector, lsst_tract, lsst_patch, lsst_band,s_ra, s_dec, t_min, t_max, s_region, access_url
FROM ivoa.ObsCore
WHERE calib_level = 2
AND CONTAINS(POINT('ICRS', 37.86,6.98), s_region) = 1
AND (lsst_band = 'g' OR lsst_band = 'r' OR lsst_band = 'i')
AND t_min > 60641.00042824074 AND t_max < 60653.00042824074
ORDER BY t_min ASC

Job phase is COMPLETED
Found 17 results
  lsst_visit  lsst_band        s_ra              s_dec       
                               deg                deg        
------------- --------- ------------------ ------------------
2024112600109         r  37.75709481524148  6.998557496684243
2024112600120         g 37.853261795796755  7.089167827554839
2024112600123         i  37.91458358975246   6.85777535893368
2024112600127         i  37.74977497897163  7.003057973017909
2024112600135         i 37.750768121673076  6.990953111438631
2024112700088         g    37.758829632072  6.993950761927504
2024112700093        

## 3. Data download

Once a table of results has been obtained, data can be downloaded by calling `download_data`. When first run, this will calculate the size of the downloaded data first; set `actually_download=True` to actually download to the chosen `output_directory` (which is created if necessary). To download the first 

In [26]:
output_dir = os.path.join(os.path.expanduser('~'), 'Rubin_dl_data')
dl_files = download_data(results[0:2], output_dir)
print(dl_files)

Found 2 files with total size 215.02 MB
set actually_download=True to actually download the data
[]


Actually download the data


In [27]:
dl_files = download_data(results[0:2], output_dir, actually_download=True)
print(dl_files)

Found 2 files with total size 215.02 MB
visit_image_LSSTComCam_r_r_03_2024112600109_R22_S02_LSSTComCam_runs_DRP_DP1_DM-51335.fits
visit_image_LSSTComCam_g_g_01_2024112600120_R22_S22_LSSTComCam_runs_DRP_DP1_DM-51335.fits
['/home/tlister/Rubin_dl_data/visit_image_LSSTComCam_r_r_03_2024112600109_R22_S02_LSSTComCam_runs_DRP_DP1_DM-51335.fits', '/home/tlister/Rubin_dl_data/visit_image_LSSTComCam_g_g_01_2024112600120_R22_S22_LSSTComCam_runs_DRP_DP1_DM-51335.fits']


In [30]:
# Show files and size
status = os.system(f"ls -lh {output_dir}")

total 206M
-rw-r--r-- 1 tlister tlister 103M Jul 17 15:27 visit_image_LSSTComCam_g_g_01_2024112600120_R22_S22_LSSTComCam_runs_DRP_DP1_DM-51335.fits
-rw-r--r-- 1 tlister tlister 103M Jul 17 15:26 visit_image_LSSTComCam_r_r_03_2024112600109_R22_S02_LSSTComCam_runs_DRP_DP1_DM-51335.fits


In [33]:
# and tidy up
for file in dl_files:
    try:
        os.remove(file)
    except FileNotFoundError:
        pass
status = os.system(f"ls -lh {output_dir}")    

total 0


## 4. Image cutouts

We can cutout a region of the resulting images using `get_image_cutout()` which will return a `astropy.fits.HDUList` (see [Astropy FITS](https://docs.astropy.org/en/stable/io/fits/index.html) docs)

(There is currently no checking of whether the passed center is within the image boundaries; feel free to contribute one to the [repo](https://github.com/lsst-sssc/rsp_queries/)...)

We'll define a center for out cutout close to the reported center (`s_ra, s_dec`) of the first image of the results from above

In [41]:
cutout_center = SkyCoord(37.76, 7.0, unit='deg')
hdulist = get_image_cutout(results[0], cutout_center, 20*u.arcsec)
print(hdulist.info())

Filename: <class '_io.BytesIO'>
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     734   ()      
  1  IMAGE         1 ImageHDU       123   (201, 200)   float32   
  2  MASK          1 ImageHDU       143   (201, 200)   int32   
  3  VARIANCE      1 ImageHDU       123   (201, 200)   float32   
  4  ARCHIVE_INDEX    1 BinTableHDU     41   91R x 7C   [1J, 1J, 1J, 1J, 1J, 64A, 64A]   
  5  FilterLabel    1 BinTableHDU     28   1R x 3C   [2X, 32A, 32A]   
  6  Detector      1 BinTableHDU    115   1R x 22C   [1QA(7), 1J, 1J, 1QA(13), 1J, 1J, 1J, 1J, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1J, 1QE(0), 1QA(3), 1J]   
  7  TransformMap    1 BinTableHDU     33   19R x 5C   [1QA(10), 1QA(7), 1QA(10), 1QA(7), 1J]   
  8  ExposureSummaryStats    1 BinTableHDU     19   21R x 1C   [1QB(104746)]   
  9  Detector      2 BinTableHDU    201   16R x 38C   [3X, 1QA(3), 1J, 1J, 1J, 1J, 1D, 1D, 1D, 1D, 1J, 1QD(2), 1QA(12), 1J, 1J, 1J, 1J, 1J, 1J, 1J, 1J, 1J, 1J,

In [43]:
## Inside the RSP (or if you have the LSST Pipelines installed) this can display the image
#import lsst.afw.display as afwDisplay
#from lsst.afw.image import ExposureF
#afwDisplay.setDefaultBackend('firefly')
#afw_display = afwDisplay.Display(frame=1)
#afw_display.image(hdulist['IMAGE'])

## 4.1 Direct image download and display

`display_image()` can take a Datalink url from the `access_url` column of the results table and display it.
Let's display the last image in the results (this part should work inside RSP or outside (if you have DS9 installed))

In [44]:
disp = display_image(results[-1]['access_url'])


An instance of ds9 was found to be running before we could
start the 'xpans' name server. You will need to perform a
bit of manual intervention in order to connect this
existing ds9 to Python.

For ds9 version 5.7 and beyond, simply register the
existing ds9 with the xpans name server by selecting the
ds9 File->XPA->Connect menu option. Your ds9 will now be
fully accessible to pyds9 (e.g., it appear in the list
returned by the ds9_targets() routine).

For ds9 versions prior to 5.7, you cannot (easily) register
with xpans, but you can view ds9's File->XPA Information
menu option and pass the value associated with XPA_METHOD
directly to the Python DS9() constructor, e.g.:

    d = DS9('a000101:12345')

The good news is that new instances of ds9 will be
registered with xpans, and will be known to ds9_targets()
and the DS9() constructor.



In [46]:
## Tidy up
del(hdulist, disp, results)

NameError: name 'hdulist' is not defined